In [ ]:
import sys
import os
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

# os.environ["OPENAI_API_KEY"] = "sk-XXX"

folderToSearch = "cps"
# folderToSearch = "docs"
# documents = []
# for file in os.listdir(folderToSearch):
#     if file.endswith(".pdf"):
#         pdf_path = f"./{folderToSearch}/" + file
#         loader = PyPDFLoader(pdf_path)
#         documents.extend(loader.load())
#     elif file.endswith('.docx') or file.endswith('.doc'):
#         doc_path = f"./{folderToSearch}/" + file
#         loader = Docx2txtLoader(doc_path)
#         documents.extend(loader.load())
#     elif file.endswith('.txt'):
#         text_path = f"./{folderToSearch}/" + file
#         loader = TextLoader(text_path)
#         documents.extend(loader.load())

# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
# documents = text_splitter.split_documents(documents)

# vectordb = Chroma.from_documents(documents, embedding=OpenAIEmbeddings(), persist_directory="./data5")
# vectordb.persist()

directory = "./data5"
vectordb = Chroma(persist_directory=directory, embedding_function=OpenAIEmbeddings())


pdf_qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo"),
    vectordb.as_retriever(search_kwargs={'k': 6}),
    return_source_documents=True,
    verbose=False
)

yellow = "\033[0;33m"
green = "\033[0;32m"
white = "\033[0;39m"

chat_history = []
print(f"{yellow}---------------------------------------------------------------------------------")
print('Welcome to the DocBot. You are now ready to start interacting with your documents')
print('---------------------------------------------------------------------------------')
while True:
    query = input(f"{green}Prompt: ")
    if query == "exit" or query == "quit" or query == "q" or query == "f":
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    result = pdf_qa(
        {"question": query, "chat_history": chat_history})
    print(f"{white}Answer: " + result["answer"])
    chat_history.append((query, result["answer"]))

# chromadb


In [5]:
# !pip install chromadb
# !pip install chroma-migrate
!chroma-migrate

Traceback (most recent call last):
  File "/Users/maarunipandithurai/.pyenv/versions/venv/bin/chroma-migrate", line 5, in <module>
    from chroma_migrate.cli import run_cli
  File "/Users/maarunipandithurai/.pyenv/versions/venv/lib/python3.9/site-packages/chroma_migrate/cli.py", line 10, in <module>
    from chroma_migrate.import_clickhouse import migrate_from_clickhouse
  File "/Users/maarunipandithurai/.pyenv/versions/venv/lib/python3.9/site-packages/chroma_migrate/import_clickhouse.py", line 3, in <module>
    from chromadb.api import API
ImportError: cannot import name 'API' from 'chromadb.api' (/Users/maarunipandithurai/.pyenv/versions/venv/lib/python3.9/site-packages/chromadb/api/__init__.py)


In [7]:
import chromadb
from chromadb.config import Settings


client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
                                    persist_directory="db/"
                                ))

ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help![0m

In [ ]:
collection = client.create_collection(name="Students")

In [ ]:
student_info = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

club_info = """
The university chess club provides an outlet for students to come together and enjoy playing
the classic strategy game of chess. Members of all skill levels are welcome, from beginners learning
the rules to experienced tournament players. The club typically meets a few times per week to play casual games,
participate in tournaments, analyze famous chess matches, and improve members' skills.
"""

university_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world."""

In [ ]:
collection.add(
    documents = [student_info, club_info, university_info],
    metadatas = [{"source": "student info"},{"source": "club info"},{'source':'university info'}],
    ids = ["id1", "id2", "id3"]
)